Chapter 24

# 数据分解
Book_4《矩阵力量》 | 鸢尾花书：从加减乘除到机器学习 (第二版)

这段代码对鸢尾花数据集的特征矩阵 $X$ 进行了多种矩阵操作和分解，以便分析数据的结构和特性。首先，代码计算了 $X$ 的 **Gram 矩阵** $G = X^T X$，展示了数据样本的内积关系。接着，基于 $G$ 构造了 **余弦相似度矩阵** $C$，通过对 $G$ 中的特征进行归一化处理，使得 $C$ 中的每个元素代表样本间的相似性。随后，代码计算数据的 **质心**（均值向量） $E(X)$ 并生成 **去均值数据矩阵** $X_c = X - E(X)$，使数据中心化，以便消除均值对数据的影响。

代码进一步计算了 **协方差矩阵** $\Sigma = \frac{1}{N} X_c^T X_c$ 和 **相关矩阵** $\rho$，分别表示特征间的协方差和标准化后的相关性。

接下来，代码进行了多种矩阵分解，包括：

1. **QR 分解**：将原始矩阵 $X$ 分解为一个正交矩阵 $Q$ 和一个上三角矩阵 $R$，满足 $X = QR$。
  
2. **Cholesky 分解**：对 Gram 矩阵 $G$ 和协方差矩阵 $\Sigma$ 进行分解，得到其对应的下三角矩阵 $L$，使得 $G = LL^T$ 和 $\Sigma = LL^T$。
  
3. **特征值分解**：对 Gram 矩阵 $G$、协方差矩阵 $\Sigma$ 和相关矩阵 $\rho$ 进行特征值分解，得到其特征值（对角化的 $\Lambda$ 矩阵）和特征向量矩阵 $V$，满足 $G = V \Lambda V^T$、$\Sigma = V \Lambda V^T$ 和 $\rho = V \Lambda V^T$。

4. **奇异值分解（SVD）**：对原始数据 $X$、去均值数据 $X_c$ 和标准化数据 $Z_X$ 分别进行 SVD 分解，得到分解形式 $X = U S V^T$，其中 $U$ 和 $V$ 分别表示数据的左右奇异向量矩阵，$S$ 是包含奇异值的对角矩阵。SVD 提供了数据的主成分方向和大小，用于后续的主成分分析（PCA）或特征提取。

通过这些分解和操作，代码全面分析了数据的相似性、协方差结构、相关性、主成分和特征方向，为理解数据的内在结构提供了重要参考。

In [1]:
import numpy as np  # 导入 numpy 进行数值计算
import matplotlib.pyplot as plt  # 导入 matplotlib 用于绘图
import pandas as pd  # 导入 pandas 进行数据操作
from sklearn.datasets import load_iris  # 从 sklearn 加载鸢尾花数据集

In [2]:
from numpy.linalg import inv  # 导入 inv 函数用于矩阵求逆
from scipy.stats import zscore  # 导入 zscore 函数用于标准化
from numpy.linalg import qr  # 导入 qr 函数进行 QR 分解
from numpy.linalg import cholesky as chol  # 导入 cholesky 函数用于 Cholesky 分解
from numpy.linalg import eig  # 导入 eig 函数进行特征值分解
from numpy.linalg import svd  # 导入 svd 函数用于奇异值分解

## 加载数据

In [3]:
iris = load_iris()  # 从 sklearn 加载鸢尾花数据集

In [4]:
X = iris.data  # 特征矩阵 X
y = iris.target  # 目标标签 y

In [5]:
feature_names = ['Sepal length, x1', 'Sepal width, x2',
                 'Petal length, x3', 'Petal width, x4']  # 特征名称

## 将特征矩阵 X 转换为 DataFrame 格式

In [6]:
X_df = pd.DataFrame(X, columns=feature_names)

## 原始数据 X

In [7]:
X = X_df.to_numpy()  # 转换 DataFrame 为 numpy 数组

## Gram 矩阵 G

In [8]:
G = X.T @ X  # 计算 Gram 矩阵，G = X^T X

## 余弦相似度矩阵 C

In [9]:
# 使用特征范数计算相似度
S_norm = np.diag(np.sqrt(np.diag(G)))  # 生成缩放矩阵，对角线元素为每列特征的范数

In [10]:
C = inv(S_norm) @ G @ inv(S_norm)  # 计算余弦相似度矩阵 C

## 数据矩阵的质心 E(X)

In [11]:
E_X = X_df.mean().to_frame().T  # 计算 X 的均值，并转换为单行 DataFrame

## 数据去均值处理 X_c

In [12]:
X_c = X_df.sub(X_df.mean())  # 每列减去其均值，得到去均值矩阵 X_c

## 协方差矩阵 Sigma

In [13]:
SIGMA = X_df.cov()  # 计算协方差矩阵 SIGMA

## 相关矩阵 P

In [14]:
RHO = X_df.corr()  # 计算相关矩阵 RHO

## 数据标准化 Z_X

In [15]:
Z_X = zscore(X_df)  # 对 X 的每列标准化，使其均值为 0，标准差为 1

## QR 分解

In [16]:
Q, R = qr(X_df, mode='reduced')  # 对 X 进行 QR 分解，mode='reduced' 保留最小矩阵维度

## Cholesky 分解

In [17]:
L_G = chol(G)  # 对 Gram 矩阵 G 进行 Cholesky 分解，得到下三角矩阵 L_G

In [18]:
R_G = L_G.T  # 上三角矩阵 R_G 为 L_G 的转置

## 协方差矩阵 Sigma 的 Cholesky 分解

In [19]:
L_Sigma = chol(SIGMA)  # 对协方差矩阵 SIGMA 进行 Cholesky 分解

In [20]:
R_Sigma = L_Sigma.T  # 上三角矩阵 R_Sigma 为 L_Sigma 的转置

## Gram 矩阵 G 的特征值分解

In [21]:
Lambs_G, V_G = eig(G)  # 对 G 进行特征值分解，得到特征值 Lambs_G 和特征向量 V_G

In [22]:
Lambs_G = np.diag(Lambs_G)  # 将特征值转换为对角矩阵形式

## 协方差矩阵 Sigma 的特征值分解

In [23]:
Lambs_sigma, V_sigma = eig(SIGMA)  # 对 SIGMA 进行特征值分解

In [24]:
Lambs_sigma = np.diag(Lambs_sigma)  # 将特征值转换为对角矩阵形式

## 相关矩阵 P 的特征值分解

In [25]:
Lambs_P, V_P = eig(RHO)  # 对相关矩阵 RHO 进行特征值分解

In [26]:
Lambs_P = np.diag(Lambs_P)  # 将特征值转换为对角矩阵形式

## 原始数据 X 的 SVD 分解

In [27]:
U_X, S_X_, V_X = svd(X_df, full_matrices=False)  # 对 X 进行 SVD 分解

In [28]:
V_X = V_X.T  # 转置 V_X

In [29]:
S_X = np.diag(S_X_)  # 将奇异值转换为对角矩阵

## 去均值数据 X_c 的 SVD 分解

In [30]:
U_Xc, S_Xc, V_Xc = svd(X_c, full_matrices=False)  # 对去均值后的数据 X_c 进行 SVD 分解

In [31]:
V_Xc = V_Xc.T  # 转置 V_Xc

In [32]:
S_Xc = np.diag(S_Xc)  # 将奇异值转换为对角矩阵

## 标准化数据 Z_X 的 SVD 分解

In [33]:
U_Z, S_Z, V_Z = svd(Z_X, full_matrices=False)  # 对标准化后的数据 Z_X 进行 SVD 分解

In [34]:
V_Z = V_Z.T  # 转置 V_Z

In [35]:
S_Z = np.diag(S_Z)  # 将奇异值转换为对角矩阵